# Exercise to Clean Messy Real Estate Data

My motivation for creating this project is to primarily identify key features which influence house prices. I am currently the in the search for houses in the Bay Area for investing. If you have any leads, feel free to contact me at `kendric "dot" v "dot" ng [at] vanderbilt "dot" edu`.

I extracted residential house data from a popular real estate website during February 2020 using a third party Chrome extension web scraper. As with HTML/CSS based web scrapers, data may not be located at the same HTML/CSS tags and can either get misplaced or list!

This `Preprocessing` notebook aims to walk through the process of cleaning online real estate data.

This is part one of my three part series on real estate prices. Please go to one of the other notebooks below if you're interested:

1. `Preprocessing`: data cleaning
2. `Exploration`: business and data understanding
3. `Prediction`: model building and prediction

Let's import the necessary python packages for this analysis.

In [1]:
import numpy as np
import pandas as pd

file = './data/raw.csv'
data = pd.read_csv(file)
data.head()

,price,bed,bath,sqft,type,yearbuilt,parking,heating,cooling,area
0,"$795,000",0.25 acres,NaN,NaN,NaN,NaN,NaN,NaN,NaN,South Bay
1,"$4,399,999",5,4.0,"3,399",Single Family,2020,Attached Garage,Heat pump,Central,South Bay
2,"$1,450,000",0.57 acres,NaN,NaN,NaN,NaN,NaN,NaN,NaN,South Bay
3,"$90,000","4,926 sqft",NaN,NaN,NaN,NaN,NaN,NaN,NaN,South Bay
4,"$225,000","5,401 sqft",NaN,NaN,NaN,NaN,NaN,NaN,NaN,South Bay


We will be using the total amount of data in the original set as a comparison check to the total amount of data lost during the preprocessing stage. It's not the best metric but for our purposes is a decent heuristic.

In [2]:
ORIGINAL_DATA_LENGTH = data.shape[0]
data.dtypes

price         object
bed           object
bath         float64
sqft          object
type          object
yearbuilt     object
parking       object
heating       object
cooling       object
area          object
dtype: object

If you are familiar with real estate data, the columns which I extracted are pretty self explanatory. The only column that I want to explain a bit more is `area`.

I extracted the real estate information using ZIP codes as the keywork search term. However, ZIP codes are plentiful and may not be useful for the purposes of training them for a linear regression model.

Hence, I manually created the `area` column based on the popular designations/districts that locals assign, i.e.

- San Francisco;
- South Bay (from South San Francisco to Palo Alto); and
- East Bay (from Oakland to Fremont).

Note that I have excluded certain cities, e.g. San Jose and Milpitas (South Bay). These are personally motivated as I will not seek to purchase houses which are too far away from the BART public transit system.

# 1. Compartmentalize Data into the Right Columns

There are a few columns in which more than one piece of information are contained within one column. There are others which are not in the correct columns. 

Let's process them so that we can have columns which properly separate the information that we need for later!

## a. `parking`

In [3]:
data['parking'].value_counts().head(15)

2 spaces                      663
No Data                       562
Attached Garage               531
1 space                       376
Detached Garage               337
3 spaces                       85
Carport                        82
None                           45
4 spaces                       42
5 spaces                       15
Off street                     14
6 spaces                       12
On street                      12
On street, Attached Garage      6
8 spaces                        5
Name: parking, dtype: int64

There are several values going on. There seems to be both numerical (i.e. the number of available spaces to park) and categorical (e.g. attached garage, carport) features. 

The number of parking spaces might prove to be an important feature to apply later on. Let's create a feature called `parking_spaces` and separate the numerical feature from the rest of the categorical features.

In [4]:
data.loc[:, 'parking_spaces'] = data['parking'].str.extract(\
                                r"(^[0-9]*)[a-zA-Z\s]*", expand=False)
data.loc[:, ['parking', 'parking_spaces']].head()

,parking,parking_spaces
0,NaN,NaN
1,Attached Garage,
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN


There are few empty strings in `parking_spaces`. Let's clean them up!

In [5]:
def change_values(column, value_desired, values_to_replace):
    """Change the values of a Dataframe Series
    
    Args:
        column - name of a pandas DataFrame column whose value to
            replace
        value_desired - value to replace the other values
        values_to_replace - a list of values to be replaced 
            or consolidated e.g. ['Single Family', 'Apartment', 'Other']

    Return:
        none
    """
    for value in values_to_replace:
        data.loc[data[column] == value, column] = value_desired

In [6]:
change_values('parking_spaces', 
              value_desired=np.nan, 
              values_to_replace=[np.nan, ''])
data[['parking', 'parking_spaces']].head()

,parking,parking_spaces
0,NaN,NaN
1,Attached Garage,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN


Much better! Let's just make sure that the information was properly transferred to `parking_spaces`.

In [7]:
has_spaces = data['parking'].str.contains('space', regex=False)\
             .replace(np.nan, False)
data.loc[has_spaces, ['parking', 'parking_spaces']].head(8)

,parking,parking_spaces
7,2 spaces,2
15,2 spaces,2
18,1 space,1
26,3 spaces,3
28,3 spaces,3
30,2 spaces,2
35,2 spaces,2
37,1 space,1


It seems that we did it correctly.

We also saw that there were categorical features in the `parking` column. Specifically, there were values with multiple categories (e.g. `Attached Garage` and `Carport` in the same entry).

For the purposes of feature engineering later, let's split them up for now into separate columns!

In [8]:
parking_info = data['parking'].str.split(', ', n=2, expand=True)
data.loc[:, 'parking1'] = parking_info[0]
data.loc[:, 'parking2'] = parking_info[1]
data.loc[:, 'parking3'] = parking_info[2]

In [9]:
data['parking1'].value_counts().head(8)

2 spaces           663
No Data            562
Attached Garage    531
1 space            376
Detached Garage    337
Carport             93
3 spaces            85
None                46
Name: parking1, dtype: int64

In [10]:
data['parking2'].value_counts()

Attached Garage    15
Detached Garage     8
On street           5
Off street          1
Name: parking2, dtype: int64

In [11]:
data['parking3'].value_counts()

Attached Garage    2
On street          1
Detached Garage    1
Name: parking3, dtype: int64

The `parking1` column still has information about parking spaces that we've already transferred into `parking_spaces`. Let's remove this duplicative information!

In [12]:
data.loc[has_spaces, ['parking1']] = np.nan
data['parking1'].value_counts().head(8)

No Data            562
Attached Garage    531
Detached Garage    337
Carport             93
None                46
Off street          25
On street           18
Garage               1
Name: parking1, dtype: int64

We seemed to have cleaned out information already in `parking_spaces`. However there are still missing information that we can clean. Note that since we scraped this from the web, let's firstly make sure that information in `parking` wasn't lost in other columns.

In [13]:
no_parking_info = data['parking1'].str.contains('No', regex=False)\
                  .replace(np.nan, True)
data.loc[no_parking_info, :].head(8)

,price,bed,bath,sqft,type,yearbuilt,parking,heating,cooling,area,parking_spaces,parking1,parking2,parking3
0,"$795,000",0.25 acres,NaN,NaN,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN
2,"$1,450,000",0.57 acres,NaN,NaN,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN
3,"$90,000","4,926 sqft",NaN,NaN,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN
4,"$225,000","5,401 sqft",NaN,NaN,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN
5,"$350,000","7,448 sqft",NaN,NaN,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN
6,"$135,000",1.52 acres,NaN,NaN,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN
7,"$2,298,888",4,3.0,"3,050",Single Family,1955,2 spaces,Forced air,None,South Bay,2,NaN,None,None
9,"$275,000","6,250 sqft",NaN,NaN,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN


There don't seem to be `parking` information in other columns. Therefore let's go ahead and clean them out!

In [14]:
change_values('parking1', 
              value_desired=np.nan, 
              values_to_replace=['No', 'No Data', 'None'])
data['parking1'].value_counts()

Attached Garage    531
Detached Garage    337
Carport             93
Off street          25
On street           18
Garage               1
Name: parking1, dtype: int64

There was also one entry that was called `Garage`. Since most garages in the United States are attached garages, for the purposes of this exercise, I'll just impute this value as `Attached garage`.

In [15]:
change_values('parking1', 
              value_desired='Attached Garage', 
              values_to_replace=['Garage'])
data['parking1'].value_counts()

Attached Garage    532
Detached Garage    337
Carport             93
Off street          25
On street           18
Name: parking1, dtype: int64

Finally, let's compare the values in the original dataset to the newly created features. This will be an indicator later on as to the predictive power of a particular feature.

In [16]:
def check_data_loss(columns_old, columns_new):

    count_new = 0
    for column in columns_new:
        count_new += data[column].count()
        print("For the category `{}`: ".format(column))
        
    try:
        count_average = count_new / len(columns_new)
    except:
        count_average = len(columns_new)
    
    # output message
    print("{:.1f}% of the data was retained after cleaning!".format\
         (count_average / ORIGINAL_DATA_LENGTH * 100))

In [17]:
check_data_loss(columns_old=['parking'], \
                columns_new=['parking_spaces', 'parking1'])

For the category `parking_spaces`: 
For the category `parking1`: 
36.2% of the data was retained after cleaning!


During the prediction phase, we will have to take note of this and determine whether or not this will be a good feature to use.

In [18]:
# random slice
data.loc[2031:2040, ['parking', 'parking_spaces', 'parking1', 'parking2', 'parking3']]

,parking,parking_spaces,parking1,parking2,parking3
2031,NaN,NaN,NaN,NaN,NaN
2032,Attached Garage,NaN,Attached Garage,None,None
2033,3 spaces,3,NaN,None,None
2034,1 space,1,NaN,None,None
2035,2 spaces,2,NaN,None,None
2036,NaN,NaN,NaN,NaN,NaN
2037,Detached Garage,NaN,Detached Garage,None,None
2038,2 spaces,2,NaN,None,None
2039,NaN,NaN,NaN,NaN,NaN
2040,Detached Garage,NaN,Detached Garage,None,None


## b. `heating`

In [19]:
data['heating'].value_counts().head(12)

No Data                  1218
Forced air               1182
Other                     109
Gas                        73
Wall                       72
Radiant                    40
Baseboard                  39
Electric                   22
Heat pump                  19
Forced air, Radiant        16
Electric, Gas               7
Baseboard, Forced air       6
Name: heating, dtype: int64

As with `parking`, there seems to be many categories under heating. However, unlike with `parking`, it seems that a dominant majority of houses in the Bay Area use `Forced air` which is also the predominant way of heating houses in the United States.

Hence, instead of splitting the values like with `parking`, we will set `Forced air` as the primary source of heating, and all others will be set to `Other`.

However, before we deal with that, let's clean up one of the values `Contact manager` which likely got stranded during our web scraping exercise!

In [20]:
data[data['heating'] == 'Contact manager']

,price,bed,bath,sqft,type,yearbuilt,parking,heating,cooling,area,parking_spaces,parking1,parking2,parking3
1137,"$13,900",3,NaN,NaN,Contact manager,Contact manager,No,Contact manager,Contact manager,San Francisco,NaN,NaN,None,None


It seems that this particular row is quite faulty data for our purposes. Hence let's go ahead and drop this row!

In [21]:
data.drop(1137, axis=0, inplace=True)
data['heating'].value_counts().head(12)

No Data                  1218
Forced air               1182
Other                     109
Gas                        73
Wall                       72
Radiant                    40
Baseboard                  39
Electric                   22
Heat pump                  19
Forced air, Radiant        16
Electric, Gas               7
Baseboard, Forced air       6
Name: heating, dtype: int64

We can also clean out the null information in `heating`. Again, let's make sure we don't lose out information in other columns.

In [22]:
no_heating_info = data['heating'].str.contains('No', regex=False)\
                  .replace(np.nan, True)
data.loc[no_heating_info, :].head(8)

,price,bed,bath,sqft,type,yearbuilt,parking,heating,cooling,area,parking_spaces,parking1,parking2,parking3
0,"$795,000",0.25 acres,NaN,NaN,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN
2,"$1,450,000",0.57 acres,NaN,NaN,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN
3,"$90,000","4,926 sqft",NaN,NaN,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN
4,"$225,000","5,401 sqft",NaN,NaN,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN
5,"$350,000","7,448 sqft",NaN,NaN,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN
6,"$135,000",1.52 acres,NaN,NaN,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN
9,"$275,000","6,250 sqft",NaN,NaN,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN
11,"$1,054,499",2,NaN,NaN,Condo,1992,Attached Garage,No Data,No Data,South Bay,NaN,Attached Garage,None,None


It doesn't seem like `heating` info will be lost either. Let's firstly remove the null information!

In [23]:
change_values('heating', 
              value_desired=np.nan, 
              values_to_replace=[np.nan, 'No', 'No Data', 'None'])
data['heating'].value_counts().head(12)

Forced air               1182
Other                     109
Gas                        73
Wall                       72
Radiant                    40
Baseboard                  39
Electric                   22
Heat pump                  19
Forced air, Radiant        16
Electric, Gas               7
Baseboard, Forced air       6
Forced air, Other           6
Name: heating, dtype: int64

Looking good! Now, let's make `Forced air` the primary method of heating for homes in the Bay Area!

In [24]:
def reduce_dimensions(column, primary_category):
    """Consolidates categorical values into one primary category and others"""
    # set to primary category
    has_primary_category = data[column].str.contains(primary_category, regex=False).replace(np.nan, False)
    data.loc[has_primary_category == True, column] = primary_category

    # set others to 'Other'
    not_primary_category = data[column] != primary_category
    not_null = ~data[column].isnull()
    data.loc[(not_primary_category & not_null), column] = 'Other'

In [25]:
reduce_dimensions(column='heating', primary_category='Forced air')
data['heating'].value_counts()

Forced air    1217
Other          385
Name: heating, dtype: int64

Again, let's compare the values in the original dataset to the newly created features to ensure that we didn't lose key data.

In [26]:
check_data_loss(columns_old=['heating'], \
                columns_new=['heating'])

For the category `heating`: 
52.3% of the data was retained after cleaning!


Better than `parking_spaces` and `parking1`!

## c. `cooling`

In [27]:
data['cooling'].value_counts().head(10)

No Data           998
Central           796
None              464
Single Family     149
Townhouse         141
Condo             125
Other              63
Wall               48
Central, Other     13
Central, Wall       9
Name: cooling, dtype: int64

Our approach to `cooling` will be similar to `heating`. Since most cooling methods for houses in the United States are done via central A/C, we will set our primary value as `Central`. However, let's go through the cleaning process first.

It seems that there are `type` entries which got their way into `cooling`. Let's check them out!

In [28]:
data[
    (data['cooling'] == 'Condo')         |
    (data['cooling'] == 'Single Family') |
    (data['cooling'] == 'Townhouse')
]

,price,bed,bath,sqft,type,yearbuilt,parking,heating,cooling,area,parking_spaces,parking1,parking2,parking3
74,"$1,603,995",4,4.0,"2,853",Single Family,2020,2 spaces,NaN,Single Family,South Bay,2,NaN,None,None
80,"$1,593,995",5,5.0,"3,005",Single Family,2020,2 spaces,NaN,Single Family,South Bay,2,NaN,None,None
88,"$1,580,995",5,5.0,"2,917",Single Family,2020,2 spaces,NaN,Single Family,South Bay,2,NaN,None,None
91,"$1,627,995",4,4.0,"3,028",Single Family,2020,2 spaces,NaN,Single Family,South Bay,2,NaN,None,None
93,"$1,593,995",5,5.0,"3,005",Single Family,2020,2 spaces,NaN,Single Family,South Bay,2,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2812,"$999,990",3,3.0,"1,911",Townhouse,2020,2 spaces,Forced air,Townhouse,East Bay,2,NaN,None,None
2819,"$699,990",2,3.0,"1,252",Townhouse,2020,2 spaces,Forced air,Townhouse,East Bay,2,NaN,None,None
2826,"$759,990",3,3.0,"1,583",Townhouse,2020,1 space,Forced air,Townhouse,East Bay,1,NaN,None,None
2828,"$679,990",2,3.0,"1,580",Townhouse,2020,1 space,Forced air,Townhouse,East Bay,1,NaN,None,None


The entries seem to indicate that the values in `cooling` are already in `type`. Informmation about `cooling` also cannot seen to be found in other columns. Let's go ahead and deem their data as `NaN`.

In [29]:
change_values('cooling', 
              value_desired=np.nan, 
              values_to_replace=['Condo', 'Single Family', 'Townhouse'])
data['cooling'].value_counts()

No Data                   998
Central                   796
None                      464
Other                      63
Wall                       48
Central, Other             13
Central, Wall               9
Central, Solar              4
Other, Wall                 3
Wall, None                  2
Central, Refrigeration      2
Central, None               2
Other, None                 2
Solar                       1
Evaporative                 1
Geothermal                  1
Name: cooling, dtype: int64

We can also clean out the null information in `cooling`. Again, let's make sure we don't lose out information in other columns.

In [30]:
no_cooling_info = data['cooling'].str.contains('No', regex=False)\
                  .replace(np.nan, True)
data.loc[no_cooling_info, :].head(8)

,price,bed,bath,sqft,type,yearbuilt,parking,heating,cooling,area,parking_spaces,parking1,parking2,parking3
0,"$795,000",0.25 acres,NaN,NaN,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN
2,"$1,450,000",0.57 acres,NaN,NaN,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN
3,"$90,000","4,926 sqft",NaN,NaN,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN
4,"$225,000","5,401 sqft",NaN,NaN,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN
5,"$350,000","7,448 sqft",NaN,NaN,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN
6,"$135,000",1.52 acres,NaN,NaN,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN
7,"$2,298,888",4,3.0,"3,050",Single Family,1955,2 spaces,Forced air,None,South Bay,2,NaN,None,None
9,"$275,000","6,250 sqft",NaN,NaN,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN


It doesn't seem like `heating` info will be lost either. Let's firstly remove the null information!

In [31]:
change_values('cooling', 
              value_desired=np.nan, 
              values_to_replace=[np.nan, 'No', 'No Data', 'None'])
data['cooling'].value_counts().head(8)

Central           796
Other              63
Wall               48
Central, Other     13
Central, Wall       9
Central, Solar      4
Other, Wall         3
Wall, None          2
Name: cooling, dtype: int64

Looking good! Now, let's make `Forced air` the primary method of heating for homes in the Bay Area!

In [32]:
reduce_dimensions(column='cooling', primary_category='Central')
data['cooling'].value_counts()

Central    826
Other      121
Name: cooling, dtype: int64

Again, let's compare the values in the original dataset to the newly created features to ensure that we didn't lose key data.

In [33]:
check_data_loss(columns_old=['cooling'], \
                columns_new=['cooling'])

For the category `cooling`: 
30.9% of the data was retained after cleaning!


There seems to be even fewer pieces of data about `cooling` than even `parking1` and `parking_spaces`!

## d. `bath` and `sqft`

In [34]:
data[['bed','sqft']]

,bed,sqft
0,0.25 acres,NaN
1,5,"3,399"
2,0.57 acres,NaN
3,"4,926 sqft",NaN
4,"5,401 sqft",NaN
...,...,...
3056,8,NaN
3057,--,NaN
3058,2,936
3059,NaN,NaN


With the `bed` column, lots of the values should in fact be in the `sqft` column. Let's find a way to migrate such data!

Before we proceed, I note that there are both units in `sqft` and `acre` in the `bed` column. Before we lose track of the units, let's firstly create a `units` column to track this. Note that the numbers in the `sqft` column are already in `sqft`, so let's set those first.

In [35]:
is_null_sqft = data['sqft'].isnull()
data.loc[~is_null_sqft, 'unit'] = 'sqft'
data[['bed', 'sqft', 'unit']]

,bed,sqft,unit
0,0.25 acres,NaN,NaN
1,5,"3,399",sqft
2,0.57 acres,NaN,NaN
3,"4,926 sqft",NaN,NaN
4,"5,401 sqft",NaN,NaN
...,...,...,...
3056,8,NaN,NaN
3057,--,NaN,NaN
3058,2,936,sqft
3059,NaN,NaN,NaN


Next, let's set the units which have `sqft` and `acre` in the `bed` column onto `units`!

In [36]:
has_sqft = data['bed'].str.contains('sqft', regex=False).\
           replace(np.nan, False)
has_acre = data['bed'].str.contains('acre', regex=False).\
           replace(np.nan, False)
data.loc[has_sqft, 'unit'] = 'sqft'
data.loc[has_acre, 'unit'] = 'acre'
data[['bed','sqft','unit']].head(8)

,bed,sqft,unit
0,0.25 acres,NaN,acre
1,5,"3,399",sqft
2,0.57 acres,NaN,acre
3,"4,926 sqft",NaN,sqft
4,"5,401 sqft",NaN,sqft
5,"7,448 sqft",NaN,sqft
6,1.52 acres,NaN,acre
7,4,"3,050",sqft


Finally, let's migrate the information from the `bed` column into the `sqft` column!

In [37]:
sqft_extract = data['bed'].str.extract\
               (r"(^[0-9,.]*)[a-zA-Z\s]*", expand=False)

# prevent inadvertent transfer of `bed` data
data.loc[(has_sqft | has_acre) & is_null_sqft, 'sqft'] = sqft_extract
data[['bed','sqft','unit']].head(8)

,bed,sqft,unit
0,0.25 acres,0.25,acre
1,5,"3,399",sqft
2,0.57 acres,0.57,acre
3,"4,926 sqft","4,926",sqft
4,"5,401 sqft","5,401",sqft
5,"7,448 sqft","7,448",sqft
6,1.52 acres,1.52,acre
7,4,"3,050",sqft


Since we've migrated the data, we can now remove the `sqft` information from `bed`.

In [38]:
data.loc[has_sqft | has_acre, 'bed'] = np.nan
data[['bed','sqft','unit']].head(8)

,bed,sqft,unit
0,NaN,0.25,acre
1,5,"3,399",sqft
2,NaN,0.57,acre
3,NaN,"4,926",sqft
4,NaN,"5,401",sqft
5,NaN,"7,448",sqft
6,NaN,1.52,acre
7,4,"3,050",sqft


We also need to clean up some data in `bed` and `sqft` as well.

In [39]:
change_values('bed', 
              value_desired=np.nan, 
              values_to_replace=['--'])
change_values('sqft', 
              value_desired=np.nan, 
              values_to_replace=['--'])
data[['bed','sqft','unit']].head(8)

,bed,sqft,unit
0,NaN,0.25,acre
1,5,"3,399",sqft
2,NaN,0.57,acre
3,NaN,"4,926",sqft
4,NaN,"5,401",sqft
5,NaN,"7,448",sqft
6,NaN,1.52,acre
7,4,"3,050",sqft


We're intentionally not cleaning the `sqft` column for now. We will deal with them later.

For now, let's check to see how much data we have lost due to this conversion!

In [40]:
check_data_loss(columns_old=['bed', 'sqft'], \
                columns_new=['bed', 'sqft'])

For the category `bed`: 
For the category `sqft`: 
73.0% of the data was retained after cleaning!


Awesome. This is a sign that we may be able to use these features later on.

## e. `type`

In [41]:
data['type'].value_counts()

Single Family            1535
Condo                     773
Townhouse                 222
Multi Family              212
Mobile / Manufactured      48
Multiple Occupancy         14
Apartment                  13
Cooperative                 4
Miscellaneous               3
Name: type, dtype: int64

Most of the data points here are important. However, for my real estate search, there are certain houses that I'm not that interested in buying, e.g. `Mobile / Manufactured`, `Multiple Occupancy`, `Cooperative`. 

Therefore, for this cleaning exercises, I will consolidate these three housing types into one `Miscellaneous` value.

In [42]:
change_values('type', 
              value_desired='Miscellaneous', 
              values_to_replace=['Cooperative','Mobile / Manufactured','Multiple Occupancy'])
data['type'].value_counts()

Single Family    1535
Condo             773
Townhouse         222
Multi Family      212
Miscellaneous      69
Apartment          13
Name: type, dtype: int64

I'm interested in mainly `Single Family` houses, which is why I'm tempted to lump in the other values into `Miscellaneous`. 

However, for now, I will keep them separate and will tweak this feature later during the machine learning preprocessing exercise.

Let's quickly check for data loss here as well.

In [43]:
check_data_loss(columns_old=['type'], \
                columns_new=['type'])

For the category `type`: 
92.3% of the data was retained after cleaning!


Brilliant!

## Summary

Let's look at the cleaning work that we've done so far!

In [44]:
data.head()

,price,bed,bath,sqft,type,yearbuilt,parking,heating,cooling,area,parking_spaces,parking1,parking2,parking3,unit
0,"$795,000",NaN,NaN,0.25,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,acre
1,"$4,399,999",5,4.0,"3,399",Single Family,2020,Attached Garage,Other,Central,South Bay,NaN,Attached Garage,None,None,sqft
2,"$1,450,000",NaN,NaN,0.57,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,acre
3,"$90,000",NaN,NaN,"4,926",NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,sqft
4,"$225,000",NaN,NaN,"5,401",NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,sqft


Now, it looks that the data are properly siloed into their respective columns. Now, Let's look at cleaning up individual columns.

# 2. Set the Proper Data Types

Let's deal with the easier `category` dtype first before dealing with the `float` dtype. `category` is a Pandas-native dtype which handles categories like they would in statistics.

For more information about this, please go to the link below:
https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html

In [45]:
data.dtypes

price              object
bed                object
bath              float64
sqft               object
type               object
yearbuilt          object
parking            object
heating            object
cooling            object
area               object
parking_spaces     object
parking1           object
parking2           object
parking3           object
unit               object
dtype: object

## a.  `category` dtype

The columns that we want to convert into the `category` dtype are:

- `area`;
- `cooling`;
- `heating`;
- `parking1`;
- `parking2`;
- `parking3`; and
- `type`,

We are not including `parking` as we will drop this column later.

In [46]:
categorical = [
    'area',
    'cooling',
    'heating',
    'parking1', 
    'parking2',
    'parking3', 
    'type'
]

for category in categorical:
    data[category] = data[category].astype('category')
    print("The {} column has been converted into the {} dtype!"\
          .format(category, dict(data.dtypes)[category]))

The area column has been converted into the category dtype!
The cooling column has been converted into the category dtype!
The heating column has been converted into the category dtype!
The parking1 column has been converted into the category dtype!
The parking2 column has been converted into the category dtype!
The parking3 column has been converted into the category dtype!
The type column has been converted into the category dtype!


Done! That was esay.

Let's also quickly check for data loss compared to the original dataset.

In [47]:
# for cases without data splits
categorical_simple = [
    'area',
    'cooling',
    'heating',
    'type'
]

for category in categorical_simple:
    check_data_loss(columns_old=[category], \
                    columns_new=[category])

For the category `area`: 
100.0% of the data was retained after cleaning!
For the category `cooling`: 
30.9% of the data was retained after cleaning!
For the category `heating`: 
52.3% of the data was retained after cleaning!
For the category `type`: 
92.3% of the data was retained after cleaning!


In [48]:
check_data_loss(columns_old=['parking'], \
                columns_new=['parking_spaces', 'parking1'])

For the category `parking_spaces`: 
For the category `parking1`: 
36.2% of the data was retained after cleaning!


Looks promising! It seems that data was not significantly removed from the dtype conversion.

## b. `float` dtype

The columns that we want to convert into the `float` dtype are:

- `bath`;
- `bed`;
- `parking_spaces`;
- `price`;
- `sqft`;
- `yearbuilt`.

Note that I did not decide to convert them into `int`s because Pandas handles `float`s a lot more natively. Let's see if we need to clean up these columns before converting them into `float`s!

In [49]:
columns_numeric = [
    'bath',
    'bed',
    'price',
    'sqft',
    'yearbuilt'
]

for column in columns_numeric:
    print(data[column].value_counts().head())

2.0    549
3.0    461
4.0    233
1.0    232
5.0     97
Name: bath, dtype: int64
3    842
2    719
4    546
5    210
1    207
Name: bed, dtype: int64
$699,000      27
$899,000      25
$799,000      24
$1,995,000    22
$998,000      18
Name: price, dtype: int64
1,080    14
2,330    13
2,009    11
1,440     8
1,344     8
Name: sqft, dtype: int64
2020       425
No Data     73
2019        72
1900        65
1925        44
Name: yearbuilt, dtype: int64


We have three things to handle when converting the columsn into floats:

1. Perform regex on the `price` and `sqft` columns;
2. Handle null data types in `yearbuilt`;
3. Convert columns into `float`s; and
4. Convert acreage data into sqft.

### i. Regex on `price` and `sqft`

In [50]:
columns_to_regex = ['price', 'sqft']
regex = [r'\$', r'\,']
for column in columns_to_regex:
    for r in regex:
        data.loc[:, column] = data[column].str.replace(r, '')
data['price'].head()

0     795000
1    4399999
2    1450000
3      90000
4     225000
Name: price, dtype: object

In [51]:
data['sqft'].head()

0    0.25
1    3399
2    0.57
3    4926
4    5401
Name: sqft, dtype: object

### ii. Null data on `yearbuilt`

Null information is included in this series. Let's figure out if data are going to be lost if we remove them.

In [52]:
no_yearbuilt_info = data['yearbuilt'].str.contains('No', regex=False)\
                  .replace(np.nan, True)
data.loc[no_yearbuilt_info, :].head(8)

,price,bed,bath,sqft,type,yearbuilt,parking,heating,cooling,area,parking_spaces,parking1,parking2,parking3,unit
0,795000,NaN,NaN,0.25,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,acre
2,1450000,NaN,NaN,0.57,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,acre
3,90000,NaN,NaN,4926,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,sqft
4,225000,NaN,NaN,5401,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,sqft
5,350000,NaN,NaN,7448,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,sqft
6,135000,NaN,NaN,1.52,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,acre
9,275000,NaN,NaN,6250,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,sqft
13,1888000,NaN,NaN,5314,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,sqft


Doesn't seem like it. Let's go ahead and clean this column up!

In [53]:
change_values(column='yearbuilt', 
              value_desired=np.nan, 
              values_to_replace=['No', 'No Data', 'None'])
data['yearbuilt'].value_counts()

2020    425
2019     72
1900     65
1925     44
1926     42
       ... 
1897      1
1883      1
1876      1
1850      1
1878      1
Name: yearbuilt, Length: 137, dtype: int64

Much better!

### iii. Convert columns into `float`

In [54]:
columns_numeric = [
    'bath',
    'bed',
    'parking_spaces',
    'price',
    'sqft',
    'yearbuilt'
]

for column in columns_numeric:
    data.loc[:, column] = pd.to_numeric(data[column], downcast='float')
    print("The {} column has been converted into the {} dtype!"\
          .format(column, dict(data.dtypes)[column]))

The bath column has been converted into the float32 dtype!
The bed column has been converted into the float32 dtype!
The parking_spaces column has been converted into the float32 dtype!
The price column has been converted into the float32 dtype!
The sqft column has been converted into the float32 dtype!
The yearbuilt column has been converted into the float32 dtype!


Let's check for data loss to double confirm!

In [55]:
columns1 = ['bath', 'price', 'yearbuilt']
for column in columns1:
    check_data_loss(columns_old=[column], \
                    columns_new=[column])

For the category `bath`: 
53.8% of the data was retained after cleaning!
For the category `price`: 
96.3% of the data was retained after cleaning!
For the category `yearbuilt`: 
89.9% of the data was retained after cleaning!


In [56]:
check_data_loss(columns_old=['bath', 'sqft'], \
                columns_new=['bath', 'sqft'])

For the category `bath`: 
For the category `sqft`: 
56.6% of the data was retained after cleaning!


In [57]:
check_data_loss(columns_old=['parking'], \
                columns_new=['parking_spaces', 'parking1'])

For the category `parking_spaces`: 
For the category `parking1`: 
36.2% of the data was retained after cleaning!


Success!

### iv. Convert acreage into sqft

In [58]:
data[['bed','sqft','unit']].head()

,bed,sqft,unit
0,NaN,0.25,acre
1,5.0,3399.00,sqft
2,NaN,0.57,acre
3,NaN,4926.00,sqft
4,NaN,5401.00,sqft


Units here are inconsistent. Let's convert all acres into sqft!

In [59]:
ACRE_TO_SQFT = 43560
data.loc[has_acre, 'sqft'] = data['sqft'] * ACRE_TO_SQFT
data[['bed','sqft','unit']].head()

,bed,sqft,unit
0,NaN,10890.000000,acre
1,5.0,3399.000000,sqft
2,NaN,24829.199219,acre
3,NaN,4926.000000,sqft
4,NaN,5401.000000,sqft


Check for data loss again here!

In [60]:
check_data_loss(columns_old=['bath', 'sqft'], \
                columns_new=['bath', 'sqft'])

For the category `bath`: 
For the category `sqft`: 
56.6% of the data was retained after cleaning!


# 3. Handling Missing Values

Let's look at how far we have come so far!

In [61]:
data.head()

,price,bed,bath,sqft,type,yearbuilt,parking,heating,cooling,area,parking_spaces,parking1,parking2,parking3,unit
0,795000.0,NaN,NaN,10890.000000,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,acre
1,4399999.0,5.0,4.0,3399.000000,Single Family,2020.0,Attached Garage,Other,Central,South Bay,NaN,Attached Garage,NaN,NaN,sqft
2,1450000.0,NaN,NaN,24829.199219,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,acre
3,90000.0,NaN,NaN,4926.000000,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,sqft
4,225000.0,NaN,NaN,5401.000000,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,sqft


I note that there are lots of missing information in this dataset. This is because some of the posts on this website have information purposefully left out. Let's look at how we can handle these cases. 

## a. Missing Row Information

Rows with mostly missing information don't add much predictive value, so we should go ahead and remove them.

### i. `price`

In [62]:
data[data['price'].isnull()].head()

,price,bed,bath,sqft,type,yearbuilt,parking,heating,cooling,area,parking_spaces,parking1,parking2,parking3,unit
17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,NaN
61,NaN,5.0,NaN,NaN,Apartment,1950.0,Attached Garage,NaN,NaN,South Bay,NaN,Attached Garage,NaN,NaN,NaN
122,NaN,4.0,NaN,NaN,Single Family,1965.0,Attached Garage,Other,NaN,South Bay,NaN,Attached Garage,NaN,NaN,NaN
183,NaN,3.0,NaN,NaN,Single Family,1937.0,Detached Garage,NaN,NaN,South Bay,NaN,Detached Garage,NaN,NaN,NaN
194,NaN,2.0,NaN,NaN,Apartment,1961.0,Carport,Forced air,NaN,South Bay,NaN,Carport,NaN,NaN,NaN


We can't predict price if there's no price information!

In [63]:
data.dropna(subset=['price'], inplace=True)
data[data['price'].isnull()].head()

,price,bed,bath,sqft,type,yearbuilt,parking,heating,cooling,area,parking_spaces,parking1,parking2,parking3,unit


We've successfully removed all null price data. Now let's drop the rows if they have neither information on `sqft` nor `type`!

### ii. `sqft` and `type`

In [64]:
data[data['sqft'].isnull() & data['type'].isnull()].head()

,price,bed,bath,sqft,type,yearbuilt,parking,heating,cooling,area,parking_spaces,parking1,parking2,parking3,unit
2017,653348.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,East Bay,NaN,NaN,NaN,NaN,NaN
2199,1549995.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,East Bay,NaN,NaN,NaN,NaN,NaN
2489,630000.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,East Bay,NaN,NaN,NaN,NaN,NaN


It seems that rows without information about `sqft` and `type` don't give us much information at all about the house! 

In [65]:
data.dropna(subset=['sqft', 'type'], how='all', inplace=True)
data[data['sqft'].isnull()].head()

,price,bed,bath,sqft,type,yearbuilt,parking,heating,cooling,area,parking_spaces,parking1,parking2,parking3,unit
11,1054499.0,2.0,NaN,NaN,Condo,1992.0,Attached Garage,NaN,NaN,South Bay,NaN,Attached Garage,NaN,NaN,NaN
16,1968000.0,5.0,NaN,NaN,Single Family,1990.0,Attached Garage,Forced air,NaN,South Bay,NaN,Attached Garage,NaN,NaN,NaN
21,2728858.0,4.0,NaN,NaN,Single Family,1920.0,Detached Garage,Other,NaN,South Bay,NaN,Detached Garage,NaN,NaN,NaN
22,1942900.0,2.0,NaN,NaN,Single Family,1925.0,Detached Garage,Other,NaN,South Bay,NaN,Detached Garage,NaN,NaN,NaN
30,3065797.0,3.0,NaN,NaN,Single Family,1954.0,2 spaces,Other,NaN,South Bay,2.0,NaN,NaN,NaN,NaN


In [66]:
data[data['type'].isnull()].head()

,price,bed,bath,sqft,type,yearbuilt,parking,heating,cooling,area,parking_spaces,parking1,parking2,parking3,unit
0,795000.0,NaN,NaN,10890.000000,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,acre
2,1450000.0,NaN,NaN,24829.199219,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,acre
3,90000.0,NaN,NaN,4926.000000,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,sqft
4,225000.0,NaN,NaN,5401.000000,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,sqft
5,350000.0,NaN,NaN,7448.000000,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,sqft


Note that we are not deleting rows with missing just one of `sqft` or `type`. Both have to missing before we drop that row. Now let's look at how many rows we dropped as a result.

In [67]:
columns1 = ['price', 'type', 'sqft']
for column in columns1:
    check_data_loss(columns_old=[column], \
                    columns_new=[column])

For the category `price`: 
96.2% of the data was retained after cleaning!
For the category `type`: 
90.6% of the data was retained after cleaning!
For the category `sqft`: 
59.0% of the data was retained after cleaning!


Promising. It seems that not a lot of data was lost compared to before the data drop.

Now let's look at the remaining columns!

### iii. Other columns

In [68]:
columns2 = ['bed', 'bath', 'yearbuilt', 'heating', 'cooling', 'parking_spaces', 'parking1']
for column in columns2:
    check_data_loss(columns_old=[column], \
                    columns_new=[column])

For the category `bed`: 
85.0% of the data was retained after cleaning!
For the category `bath`: 
53.5% of the data was retained after cleaning!
For the category `yearbuilt`: 
88.3% of the data was retained after cleaning!
For the category `heating`: 
51.9% of the data was retained after cleaning!
For the category `cooling`: 
30.7% of the data was retained after cleaning!
For the category `parking_spaces`: 
39.3% of the data was retained after cleaning!
For the category `parking1`: 
31.9% of the data was retained after cleaning!


So, based on this, it seems that the `bed`, `bath`, `yearbuilt` and `heating` columns have at least a majority of the data there. Let's look at them!

In [69]:
data[data['bed'].isnull() & data['bath'].isnull()].head()

,price,bed,bath,sqft,type,yearbuilt,parking,heating,cooling,area,parking_spaces,parking1,parking2,parking3,unit
0,795000.0,NaN,NaN,10890.000000,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,acre
2,1450000.0,NaN,NaN,24829.199219,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,acre
3,90000.0,NaN,NaN,4926.000000,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,sqft
4,225000.0,NaN,NaN,5401.000000,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,sqft
5,350000.0,NaN,NaN,7448.000000,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,sqft


In [70]:
data[data['yearbuilt'].isnull()].head()

,price,bed,bath,sqft,type,yearbuilt,parking,heating,cooling,area,parking_spaces,parking1,parking2,parking3,unit
0,795000.0,NaN,NaN,10890.000000,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,acre
2,1450000.0,NaN,NaN,24829.199219,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,acre
3,90000.0,NaN,NaN,4926.000000,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,sqft
4,225000.0,NaN,NaN,5401.000000,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,sqft
5,350000.0,NaN,NaN,7448.000000,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,sqft


In [71]:
data[data['heating'].isnull()].head()

,price,bed,bath,sqft,type,yearbuilt,parking,heating,cooling,area,parking_spaces,parking1,parking2,parking3,unit
0,795000.0,NaN,NaN,10890.000000,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,acre
2,1450000.0,NaN,NaN,24829.199219,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,acre
3,90000.0,NaN,NaN,4926.000000,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,sqft
4,225000.0,NaN,NaN,5401.000000,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,sqft
5,350000.0,NaN,NaN,7448.000000,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,sqft


It seems that for all of rows with missing data from the above 4 columns, I can see that all of them have `sqft`. Before I can throw this out, I need to check later whether or not `sqft` has predictive power for the model later.

## b. Imputing values (pushed to prediction stage)

The next strategy that we can use is imputing values. This may dilute the predictive power of a model. However, this can result in having fewer issues with building the model. I propose to push this to the prediction phase of the model, as we have more powerful tools that we can use then.

# 4. Final Check

Now that we have performed most of the cleaning exercises. Let's check if there's anything else that we need to do before export this dataset for analysis!

In [72]:
data.head()

,price,bed,bath,sqft,type,yearbuilt,parking,heating,cooling,area,parking_spaces,parking1,parking2,parking3,unit
0,795000.0,NaN,NaN,10890.000000,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,acre
1,4399999.0,5.0,4.0,3399.000000,Single Family,2020.0,Attached Garage,Other,Central,South Bay,NaN,Attached Garage,NaN,NaN,sqft
2,1450000.0,NaN,NaN,24829.199219,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,acre
3,90000.0,NaN,NaN,4926.000000,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,sqft
4,225000.0,NaN,NaN,5401.000000,NaN,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN,sqft


We have some duplicate columns that we can delete here.

In [73]:
data.drop(['parking', 'unit'], axis=1, inplace=True)
data.head()

,price,bed,bath,sqft,type,yearbuilt,heating,cooling,area,parking_spaces,parking1,parking2,parking3
0,795000.0,NaN,NaN,10890.000000,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN
1,4399999.0,5.0,4.0,3399.000000,Single Family,2020.0,Other,Central,South Bay,NaN,Attached Garage,NaN,NaN
2,1450000.0,NaN,NaN,24829.199219,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN
3,90000.0,NaN,NaN,4926.000000,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN
4,225000.0,NaN,NaN,5401.000000,NaN,NaN,NaN,NaN,South Bay,NaN,NaN,NaN,NaN


And we're finished. Let's go ahead and export the data!

In [74]:
data.to_csv('./zillow/zillow_clean.csv', index=False)

# Further Consideration

## 1. Scalability Issues

For the purposes of this project, it was okay to brute force some of the preprocessing tasks. However, having lots of missing information that is not `price`, `sqft` or `area` will make it difficult to scale the scraping process using just a web scraping tool.

Also, there were stranded values that ended up in other columns. Who knows? There may have been values that the web scraper didn't properly pick up!

This shows the difficulty of scaling this type of cleaning process for a higher production scale project.

## 2. Mutual Exclusivity of Parking and Parking Spaces

Lots of parking spaces that have been marked in the dataset may in fact have parking types in `parking`. However, since the data treats them as separate features, it makes it more difficult for us to infer between the two features. We will have to impute some values later in the `Prediction` notebook to have a guess as to the relationship between these two features.

## 3. dtypes

In preparing the preprocessing tasks, I found two dtypes to be troublesome:

### a. Categorical Features

Categorical - when using pandas, it seemed easier to leave them as `object` dtype and then convert them into the `category` dtype after preprocessing. Converting it into `string` raised some other issues (e.g. NAtype string versus np.nan in an array). 

### b. Numerical Features

Numerical - as I found out the hard way, pandas more natively handles `float` dtypes than `int` dtypes, and even within float, there are different `float` types to manage across packages (e.g. Python native, numpy, pandas). It's good to be consistent across the notebook for exploratory purposes. However, having everything be represented as `float` dtypes may make it difficult to take this code into production (e.g. memory handling).

## 4. Reducing Dimensionality

Decisions like simplifying `heating` to be `Forced air`, `Other` and `NaN` can prove to be high risk feature engineering tactics. For the purposes of this project, they can be seen as practical and common sense. If more time were to be alloted, I would experiment more with the contribution of reduced dimensions towards improvement in model performance.

## 5. Duplicity of Functions across Packages

As I alluded to in number three regarding dtypes, data can be treated differently across standard Python, numpy and pandas. For instance, string manipulation doesn't apply exactly in the standard Python string libraries and pandas when it comes to regex (regular expression) search and replace.

Besides consistency, performance tradeoffs also have to be considered when relying heavily on pandas packages.

# Next Steps

Now that I've processed the data to be usable for exploration, we will now go through the standard CRISP-DM process in data science to understand what are the biggest contributors to housing prices.